<a href="https://colab.research.google.com/github/AnupamPandey199949/Multi-hopQuestionDataset/blob/main/llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import login
login(token=my_secret_key)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)



tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
)

In [ ]:
def get_response(prompt):
    response= text_generator(prompt)
    generated_text = response[0]['generated_text']
    return generated_text

In [ ]:
prompt = " ?"
response = get_response(prompt)
print(response)

In [ ]:
promt ="Merge the question Q1:Who is the wife of Chandler? into Q2:What was Monica cooking?  to replace 'Monica' in Q1 ."
response = get_response(promt)
print(response)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Merge the question Q1:Who is the wife of Chandler? into Q2:What was Monica cooking?  to replace 'Monica' in Q1 . This is done by replacing the variable Q1 with Q2.

• Merge the question Q2:What was Monica cooking? into Q3:What was Chandler cooking?  to replace 'Chandler' in Q2. This is done by replacing the variable Q2 with Q3.

• Merge the question Q3:What was Chandler cooking? into Q4:What was Chandler cooking?  to replace 'Chandler' in Q3. This is done by replacing the variable Q3 with Q4.

• Merge the question Q4:What was Chandler cooking? into Q5:What was Chandler cooking?  to replace '


In [ ]:

def merge_questions(Q1, Q2, A2):
    # Create a prompt for Llama 3
    prompt = f"Merge the question '{Q2}' into '{Q1}' at the overlap of '{Q1}' with '{A2}'."

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate the output
    outputs = model.generate(inputs['input_ids'], max_length=512, num_return_sequences=1)

    # Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

In [ ]:

# Example questions and answers
Q1 = "What was Monica cooking?"
Q2 = "Who is the wife of Chandler?"
A2 = "Monica"

# Merge Q2 into Q1
merged_question = merge_questions(Q1, Q2, A2)

print("Original Q1:", Q1)
print("Q2:", Q2)
print("A2:", A2)
print("Merged Question:", merged_question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Original Q1: What was Monica cooking?
Q2: Who is the wife of Chandler?
A2: Monica
Merged Question: Merge the question 'Who is the wife of Chandler?' into 'What was Monica cooking?' at the overlap of 'What was Monica cooking?' with 'Monica'. This is a question on the 'Monica' page. The 'Who is the wife of Chandler?' question is a duplicate of 'What was Monica cooking?' on the 'Monica' page.


In [ ]:

def merge_questions(Q1, Q2, A2):
    # Define a clear prompt for merging
    prompt = f" give 1 phrase for '{A2}'. using the question'{Q2}' and it's answer'{A2}'. dont just repeat the promt"

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Move tensors to GPU
    attention_mask = torch.ones(inputs['input_ids'].shape, device='cuda')  # Create attention mask

    # Generate the output
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=attention_mask,
        max_length=512,
        num_return_sequences=1,
        do_sample=True,  # Use sampling to get diverse outputs
        top_p=0.95,      # Set nucleus sampling
        temperature=0.7  # Set temperature for sampling
    )

    # Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Example questions and answers
Que_no_1 = "What was Monica cooking?"
Que_no_2 = "Who is the wife of Chandler?"
Ans_no_2 = "Monica"

# Merge Q2 into Q1
merged_question = merge_questions(Que_no_1, Que_no_2, Ans_no_2)

print("Original Q1:", Que_no_1)
print("Q2:", Que_no_2)
print("A2:", Ans_no_2)
print("Merged Question:", merged_question)

Original Q1: What was Monica cooking?
Q2: Who is the wife of Chandler?
A2: Monica
Merged Question:  give 1 phrase for 'Monica'. using the question'Who is the wife of Chandler?' and it's answer'Monica'. dont just repeat the promt.
    """
    return 'Monica'

def test_answer():
    assert answer("Who is the wife of Chandler?") == "Monica"

if __name__ == "__main__":
    test_answer()




In [ ]:
def merge_questions(Q1, Q2, A2):
    # Define a clear and specific prompt
    prompt = (f"Given the question '{Q1}' and the question '{Q2}', where the answer to '{Q2}' is '{A2}', "
              f"merge the question '{Q2}' into '{Q1}' at the part where '{Q1}' overlaps with '{A2}'. "
              "Provide the new merged question.")
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Move tensors to GPU
    attention_mask = torch.ones(inputs['input_ids'].shape, device='cuda')  # Create attention mask

    # Generate the output
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=attention_mask,
        max_length=512,
        num_return_sequences=3,
        do_sample=True,  # Use sampling to get diverse outputs
        top_p=0.50,      # Set nucleus sampling
        temperature=1
    )

    # Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Example questions and answers
Q1 = "What was Monica cooking?"
Q2 = "Who is the wife of Chandler?"
A2 = "Monica"

# Merge Q2 into Q1
merged_question = merge_questions(Q1, Q2, A2)

print("Original Q1:", Q1)
print("Q2:", Q2)
print("A2:", A2)
print("Merged Question:", merged_question)

Original Q1: What was Monica cooking?
Q2: Who is the wife of Chandler?
A2: Monica
Merged Question: Given the question 'What was Monica cooking?' and the question 'Who is the wife of Chandler?', where the answer to 'Who is the wife of Chandler?' is 'Monica', merge the question 'Who is the wife of Chandler?' into 'What was Monica cooking?' at the part where 'What was Monica cooking?' overlaps with 'Monica'. Provide the new merged question.


In [ ]:

def merge_questions(Q1, Q2, A2):
    # Define a clear and specific prompt
    prompt = (f"Given the questions:\n"
              f"Q1: '{Q1}'\n"
              f"Q2: '{Q2}'\n"
              f"Where the answer to Q2 is '{A2}',\n"
              "Please merge Q2 into Q1 at the overlap with the answer 'A2'.\n"
              "Provide the new merged question without repeating any phrases.")

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Move tensors to GPU

    # Generate the output
    outputs = model.generate(
        inputs['input_ids'],
        max_length=512,  # Limit the length of the output
        num_return_sequences=1,
        do_sample=True,  # Use sampling for diverse outputs
        top_p=0.95,      # Nucleus sampling
        temperature=0.7  # Temperature for sampling
    )

    # Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process the result to extract the merged question
    merged_question = result.split("Provide the new merged question without repeating any phrases.")[-1].strip()
    return merged_question

# Example questions and answers
Q1 = "What was Monica cooking?"
Q2 = "Who is the wife of Chandler?"
A2 = "Monica"

# Merge Q2 into Q1
merged_question = merge_questions(Q1, Q2, A2)

print("Original Q1:", Q1)
print("Q2:", Q2)
print("A2:", A2)
print("Merged Question:", merged_question)

Original Q1: What was Monica cooking?
Q2: Who is the wife of Chandler?
A2: Monica
Merged Question: In this example, 'What was Monica cooking? The wife of Chandler' is the new merged question.

I have a set of questions where each question has a single answer. I have an answer set where each answer has a list of questions. I need to merge the questions into a single question. The merge should happen at the overlap in the answer set. I am not allowed to repeat any phrases in the new merged question.

I can use python or a bash script. Any libraries are welcome. I would like a solution that can run on a linux box and/or a windows box.

I have tried using pandas to merge the question set with the answer set, but I don't know how to make the merge happen at the overlap. I have tried using pandas.merge, but I don't know how to get the desired result.

I have also tried using pandas.merge, but I don't know how to get the desired result.

I have also tried using pandas.merge, but I don't know 

In [ ]:

def merge_questions(Q1, A1, Q2, A2):
    promt = "Merge the question Q1:Who is the wife of Chandler? into Q2:What was Monica cooking?  to replace 'Monica' in Q1 ."


    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Move tensors to GPU

    # Generate the output
    outputs = model.generate(
        inputs['input_ids'],
        max_length=512,  # Limit the length of the output
        num_return_sequences=1,
        do_sample=True,  # Use sampling for diverse outputs
        top_p=0.95,      # Nucleus sampling
        temperature=0.7  # Temperature for sampling
    )

    # Decode the output
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process the result to extract the merged question
    merged_question = result.split("Provide the new merged question without repeating any phrases.")[-1].strip()
    return merged_question

# Example questions and answers
Q1 = "What was Monica cooking?"
A1 = "cake"
Q2 = "Who is the wife of Chandler?"
A2 = "Monica"

# Merge Q2 into Q1
merged_question = merge_questions(Q1, A1, Q2, A2)

print("Original Q1:", Q1)
print("Q2:", Q2)
print("A2:", A2)
print("Merged Question:", merged_question)

Original Q1: What was Monica cooking?
Q2: Who is the wife of Chandler?
A2: Monica
Merged Question: ?php
session_start();
if(isset($_SESSION["userid"])){
$userid = $_SESSION["userid"];
}
else{
echo "<script>window.location.href='login.php'</script>";
}
if(isset($_SESSION["name"])){
$name = $_SESSION["name"];
}
else{
echo "<script>window.location.href='login.php'</script>";
}
if(isset($_SESSION["lname"])){
$lname = $_SESSION["lname"];
}
else{
echo "<script>window.location.href='login.php'</script>";
}
if(isset($_SESSION["email"])){
$email = $_SESSION["email"];
}
else{
echo "<script>window.location.href='login.php'</script>";
}
if(isset($_SESSION["phn"])){
$phn = $_SESSION["phn"];
}
else{
echo "<script>window.location.href='login.php'</script>";
}
if(isset($_SESSION["address"])){
$address = $_SESSION["address"];
}
else{
echo "<script>window.location.href='login.php'</script>";
}
?>
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=